In [1]:
import anndata
from scdesigner.minimal.simulator import scdesigner
from scdesigner.minimal.negbin import NegBin
from scdesigner.minimal.standard_covariance import StandardCovariance

sim = scdesigner(
    NegBin("~ bs(pseudotime, df=5)"),
    StandardCovariance("~ -1 + cell_type")
)

example_sce = anndata.read_h5ad("data/example_sce.h5ad")
sim.fit(example_sce, max_epochs=1, lr=0.01)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 700    | train
------------------------------------------------
700       Trainable params
0         Non-trainable params
700       Total params
0.003     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  1.87it/s, v_num=789, train_loss_step=1.77e+4, train_loss_epoch=4.57e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 3/3 [00:01<00:00,  1.87it/s, v_num=789, train_loss_step=1.77e+4, train_loss_epoch=4.57e+5]


Estimating copula covariance: 100%|██████████| 3/3 [00:01<00:00,  1.80it/s]


In [2]:
sim.predict(example_sce.obs[:10])

{'mean': array([[1.0107875 , 1.0136665 , 1.0017115 , 1.0134748 , 0.9994596 ,
         1.0143161 , 1.0108783 , 1.0112027 , 1.0251377 , 0.9993954 ,
         0.9996869 , 0.9890562 , 0.9952399 , 1.007602  , 1.0026752 ,
         1.023669  , 1.0018021 , 1.0209134 , 1.0111542 , 1.013545  ,
         1.0107422 , 1.0100027 , 1.0104778 , 0.99341595, 1.0160089 ,
         0.9882907 , 1.0038074 , 1.0126884 , 1.017372  , 1.0205735 ,
         0.99958086, 0.9857477 , 0.98628426, 1.0119799 , 1.0120121 ,
         0.9935525 , 1.0104072 , 1.0129973 , 0.9921745 , 1.0063559 ,
         0.98244995, 0.99087584, 1.0143988 , 0.99804807, 1.0237828 ,
         1.0216992 , 0.9940934 , 0.99637026, 1.0219686 , 1.011362  ,
         1.0179238 , 1.0038136 , 1.0029312 , 0.9921099 , 1.000397  ,
         1.0003233 , 0.9935467 , 1.004193  , 1.0132382 , 0.9932072 ,
         0.9793075 , 1.0026793 , 1.0384319 , 1.0119886 , 1.0195576 ,
         1.0110116 , 0.97965306, 1.025485  , 1.0027183 , 0.99237275,
         1.0246172 , 1.013

In [3]:
sim.sample(example_sce.obs[:10])

AnnData object with n_obs × n_vars = 10 × 100
    obs: 'clusters_coarse', 'clusters', 'S_score', 'G2M_score', 'cell_type', 'sizeFactor', 'pseudotime'

In [4]:
sim.complexity()

Computing log-likelihood...: 100%|██████████| 3/3 [00:01<00:00,  1.61it/s]


{'aic': np.float64(-49788.486611269705), 'bic': np.float64(61952.47494885887)}

In [5]:
sim.parameters

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
  Intercept               -0.000535  0.013939  0.001546  0.005087 -0.005308   
  bs(pseudotime, df=5)[1]  0.008650 -0.001619 -0.017115 -0.018686  0.010720   
  bs(pseudotime, df=5)[2] -0.000378  0.018105 -0.003796  0.010845  0.004892   
  bs(pseudotime, df=5)[3]  0.014522 -0.006684 -0.002653  0.007668  0.004217   
  bs(pseudotime, df=5)[4]  0.009932  0.004754  0.009954  0.008376  0.006126   
  bs(pseudotime, df=5)[5] -0.019148  0.011611  0.010253  0.017118 -0.003216   
  
                               Chga       Cck      Ins1      Nnat      Ins2  \
  Intercept                0.013160 -0.001982  0.016249  0.021854  0.000936   
  bs(pseudotime, df=5)[1] -0.001557 -0.042011  0.011663 -0.005585  0.007257   
  bs(pseudotime, df=5)[2] -0.000941  0.020505 -0.021501  0.002518  0.010000   
  bs(pseudotime, df=5)[3]  0.002060  0.016419 -0.000110  0.003602 -0.006483   
  bs(pseudotime, df=5)[4] -0.

In [6]:
from scdesigner.minimal.simulator import scdesigner
from scdesigner.minimal.negbin import NegBin
from scdesigner.minimal.standard_covariance import StandardCovariance

sim = scdesigner(
    NegBin({"mean": "~ bs(pseudotime,df=5)", "dispersion": "~ pseudotime"}),
    StandardCovariance("~ -1 + cell_type")
)

sim.fit(example_sce, batch_size=int(256), max_epochs=1)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type          | Params | Mode 
------------------------------------------------
0 | coefs | ParameterDict | 800    | train
------------------------------------------------
800       Trainable params
0         Non-trainable params
800       Total params
0.003     Total estimated model params size (MB)
1         Modules in train mode
0         Modules in eval mode


Epoch 0: 100%|██████████| 9/9 [00:01<00:00,  4.72it/s, v_num=790, train_loss_step=1.76e+4, train_loss_epoch=1.14e+5]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 9/9 [00:01<00:00,  4.71it/s, v_num=790, train_loss_step=1.76e+4, train_loss_epoch=1.14e+5]


Estimating copula covariance: 100%|██████████| 9/9 [00:01<00:00,  5.47it/s]


In [7]:
print(sim.parameters)
print(sim.sample())
print(sim.predict())

{'marginal': {'mean':                               Pyy      Iapp      Chgb      Rbp4      Spp1  \
Intercept                0.022061  0.002766  0.006531  0.019222  0.020967   
bs(pseudotime, df=5)[1] -0.005269 -0.008728 -0.026590 -0.009863 -0.001961   
bs(pseudotime, df=5)[2]  0.008783  0.015831  0.003302  0.003370  0.030807   
bs(pseudotime, df=5)[3]  0.022609 -0.005569  0.005983 -0.009218 -0.010498   
bs(pseudotime, df=5)[4]  0.001609  0.001710  0.008429  0.009627 -0.000693   
bs(pseudotime, df=5)[5]  0.000274  0.004627  0.018294  0.015143 -0.010973   

                             Chga       Cck      Ins1      Nnat      Ins2  \
Intercept                0.007277  0.009838  0.016021  0.012955  0.000311   
bs(pseudotime, df=5)[1] -0.002797  0.011133  0.016151 -0.005190 -0.008328   
bs(pseudotime, df=5)[2]  0.014476  0.027961 -0.013459 -0.004388 -0.001073   
bs(pseudotime, df=5)[3]  0.005600 -0.008861  0.002033  0.001557  0.000803   
bs(pseudotime, df=5)[4]  0.013658  0.017416  0.021939

In [8]:
example_sce = anndata.read_h5ad("data/million_cells.h5ad", backed=True, chunk_size=int(2e4))

sim = scdesigner(
    NegBin("~ celltype"),
    StandardCovariance("~ -1 + celltype")
)
#sim.fit(example_sce, batch_size=int(1000), lr=0.01)